# Configuration

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Imports

In [21]:
import pandas as pd
import numpy as np

import gc

from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import mean_squared_error

import scipy.sparse

import pickle

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.layers import Dense, Input, Dropout, Concatenate, BatchNormalization


dir_test_tar_index = '/content/drive/MyDrive/Comps/Multimodal_Single-Cell_Integration/Data/Multi/multi_index_sub.parquet'
dir_meta           = '/content/drive/MyDrive/Comps/Multimodal_Single-Cell_Integration/Data/metadata.parquet'


dir_train      = '/content/drive/MyDrive/Comps/Multimodal_Single-Cell_Integration/Data/Multi/train_multi_'
dir_train_tar  = '/content/drive/MyDrive/Comps/Multimodal_Single-Cell_Integration/Data/Multi/train_multi_target_'
dir_test       = '/content/drive/MyDrive/Comps/Multimodal_Single-Cell_Integration/Data/Multi/test_'
dir_pca_target      = '/content/drive/MyDrive/Comps/Multimodal_Single-Cell_Integration/Data/Multi/test_multi_target_inv_'

#dir_train = '/content/drive/MyDrive/Comps/Multimodal_Single-Cell_Integration/Data/Multi'
#dir_train = '/content/drive/MyDrive/Comps/Multimodal_Single-Cell_Integration/Data/Multi'

# Load Data

In [30]:
# Load Train data
train     = pd.read_parquet(dir_train + '/train_svd_70.parquet')

# Load meta data
metadata_df = pd.read_parquet(dir_meta)
metadata_df = metadata_df[metadata_df.technology=="multiome"]
meta        = metadata_df.reindex(train.index)
train       = train.values

# Load test data
test      = pd.read_parquet(dir_train + '/test_svd_70.parquet').values

# Load train target
train_tar = pd.read_parquet(dir_train + '/train_tar_svd_70.parquet').values

# Load test target inverse
pca_target = pd.read_parquet(dir_train + '/test_inv_svd_70.parquet')
sub_index  = pd.read_parquet(dir_test_tar_index)


dumies = pd.read_parquet('/content/drive/MyDrive/Comps/Multimodal_Single-Cell_Integration/Data/Multi/dummies_train.parquet')
train  = np.concatenate((train, dumies.values), axis=1)

dumies = pd.read_parquet('/content/drive/MyDrive/Comps/Multimodal_Single-Cell_Integration/Data/Multi/dummies_test.parquet')
test   = np.concatenate((test, dumies.values), axis=1)

in_shape  = train.shape[1]
tar_shape = train_tar.shape[1]
in_shape, tar_shape

# Model and parameters

In [31]:
# The competition metric - averaged sum of correlation coefficients
def correlation_score(y_true, y_pred):
  
    corrsum = 0
    for i in range(len(y_true)):
        corrsum += np.corrcoef(y_true[i], y_pred[i])[1, 0]
    return corrsum / len(y_true)
    
# The competition metric as a custom tf metric for monitoring during CV.
class corr_coff(tf.keras.metrics.Metric):

    def __init__(self, name="corr_coff", **kwargs):
        super(corr_coff, self).__init__(name=name, **kwargs)
        self.corr_coff = self.add_weight(name="cc", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):

        numerator   = tf.reduce_sum(tf.multiply(y_true - tf.expand_dims(tf.reduce_mean(y_true, axis=1), 1), y_pred - tf.expand_dims(tf.reduce_mean(y_pred, axis=1), 1)), axis=1)
        denominator = K.sqrt(K.var(y_true,axis=1) * K.var(y_pred,axis=1))  
        
        self.corr_coff = tf.reduce_mean(numerator/y_true.shape[1]/denominator)
    
    def result(self):
        return self.corr_coff

    def reset_state(self):
        self.corr_coff = self.add_weight(name="cc", initializer="zeros")
    
# Callback function for training 
plateau = tf.keras.callbacks.ReduceLROnPlateau(
    monitor   = "val_loss",
    factor    = 0.25,
    patience  = 3,
    verbose   = 0,
    mode      = "min",
    min_delta = 1e-7)

es = tf.keras.callbacks.EarlyStopping(
    monitor              = 'val_loss',
    min_delta            = 1e-7,
    patience             = 10,
    verbose              = 1,
    mode                 = "min",
    restore_best_weights = True)

# Model

In [28]:
def create_model():

    REG1 = tf.keras.regularizers.l2(.00001)
    REG2 = tf.keras.regularizers.l1(.000001)
  
    x0 = Input(shape=in_shape)


    x1 = Dense(units=in_shape*2, activity_regularizer=REG1, activation='selu')(x0)

    x2 = Dense(units=in_shape*3, activation='selu')(x1)
    x2 = Dropout(.1)(x2)

    x3 = Dense(units=in_shape*2, kernel_regularizer=REG1, activation='selu')(x2)

    x4 = Dense(units=in_shape*4, activation='selu')(x3)
    x4 = Dropout(.2)(x4)
    
    x5 = Dense(units=in_shape*2, activity_regularizer=REG1, activation='selu')(x4)

    x6 = Dense(units=in_shape*5, activation='selu')(x5)
    x6 = Dropout(.3)(x6)

    x7 = Dense(units=in_shape*2, kernel_regularizer=REG1, activation='selu')(x6)

    x8 = Dense(units=in_shape*6, activation='selu')(x7)
    x8 = Dropout(.4)(x8)


    x = Concatenate()([x7, x8])
    x = Dense(units=tar_shape, activity_regularizer=REG2, activation='linear',)(x)
    
    model = Model(x0, x)
    
    return model

# Training

In [ ]:
#kf = GroupKFold(n_splits=3)
tf.keras.utils.set_random_seed(12)

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold,(idx_tr, idx_va) in enumerate(kf.split(train, meta.donor)): #, groups=meta.donor)): #)): # )): #

    print(f'\n --------- FOLD {fold} -----------')
    
    X_tr = train[idx_tr]
    y_tr = train_tar[idx_tr]
    
    X_va = train[idx_va]
    y_va = train_tar[idx_va]
    
    model = create_model()

    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005),
                  loss      = 'mse',
                  metrics   = None   #[corr_coff()]
                  )
    
    model.fit(X_tr,
              y_tr,
              validation_data = (X_va, y_va),
              epochs          = 1000,
              verbose         = 1,
              batch_size      = 2*128,
              callbacks       = [es, plateau]
             )
    
    model.save(f"model_{fold}")
    
    pred = model.predict(X_va)

    '''
    # This loop calculates relevant metrics for different types of cells
    cur_meta = meta.loc[meta.idx.isin(idx_va)].copy()
    cur_meta['new_idx'] = range(cur_meta.shape[0])

    for cell in meta.cell_type.unique():

      cell_idx = cur_meta.loc[cur_meta.cell_type == cell].new_idx.values

      cur_pred = pred[cell_idx]
      cur_y_va = y_va[cell_idx]

      print(f"{cell} corr = {np.round(mean_squared_error(cur_y_va,cur_pred),4)}" , '\n')
    
    '''
    print('\n', f"Fold corr = {np.round(mean_squared_error(y_va, pred),4)}", '\n')
        
    del X_tr, X_va, y_tr, y_va
    gc.collect()


## Test prediction

In [26]:
#preds = np.zeros((test.shape[0], 23418), dtype='float16')
results = pd.DataFrame()

for fold in [0,1,2,3,4]:
    
  print(f'fold {fold} prediction')
  
  model = tf.keras.models.load_model(f"model_{fold}", compile=False)
  preds = model.predict(test)
  final = (preds@pca_target.values)
  # Due to too large data only some entries of the predicted data form part of the final submission.
  # The exact indicies of those predictions were saved in 'sub_index'
  final = final[sub_index.a.values, sub_index.b.values]

  del model, preds
  gc.collect()
  
  results[f'fold_{fold}'] = final

fold 0 prediction
1748/1748 [==============================] - 3s 1ms/step
fold 1 prediction
1748/1748 [==============================] - 3s 1ms/step
fold 2 prediction
1748/1748 [==============================] - 3s 1ms/step
fold 3 prediction
1748/1748 [==============================] - 3s 1ms/step
fold 4 prediction
1748/1748 [==============================] - 3s 1ms/step


In [30]:
results.to_parquet('/content/drive/MyDrive/Comps/Multimodal_Single-Cell_Integration/Predictions/multi.parquet')